In [0]:
#find . -name '*.jpg' -exec mv {} ../Dataset/ \;

In [0]:
import numpy as np
import pandas as pd
import glob
from PIL import Image
from tqdm import tqdm
from keras.models import load_model
import matplotlib.pyplot as plt
# import numexpr as ne
import keras_metrics

In [0]:
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Flatten, Dense, Dropout
from keras.activations import sigmoid,relu
from keras.optimizers import Adam
from sklearn.metrics import classification_report,f1_score, confusion_matrix

In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
#Normalize
def normalize(arr):
    arr=arr.astype('float32')
    if arr.max() > 1.0:
        arr/=255.0
    return arr
#Grayscale
gray = lambda rgb : np.dot(rgb[... , :3] , [0.299 , 0.587, 0.114]) 

In [0]:
# #Postives
positive = glob.glob('./Postive/*.jpg')
y = np.array([normalize(gray(np.array(Image.open(fname).resize((64,64), Image.ANTIALIAS)))) for fname in tqdm(positive)])
finalp=[]
for i in range(0,len(y)):
    finalp.append(y[i].flatten())
del y
positive_df=pd.DataFrame({'X':finalp,'y':1})

100%|██████████| 2999/2999 [00:09<00:00, 311.48it/s]


In [0]:
# #Negatives
negative = glob.glob('./Negative/*.jpg')
n = np.array([normalize(gray(np.array(Image.open(fname).resize((64,64), Image.ANTIALIAS)))) for fname in tqdm(negative[:])])
final=[]
for i in range(0,len(n)):
    final.append(n[i].flatten())
negative_df=pd.DataFrame({'X':final,'y':0})

100%|██████████| 2776/2776 [00:37<00:00, 58.54it/s]


In [0]:
# negative_df = pd.read_csv('negatives.csv')
final_dataset=pd.concat([positive_df,negative_df])
final_dataset=final_dataset.sample(frac=1)
final_dataset.reset_index(inplace=True)
final_dataset.drop('index',inplace=True,axis=1)

In [0]:
final_dataset.head()

,X,y
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
1,"[0.062231373, 0.06615294, 0.062231373, 0.06223...",1
2,"[0.69899607, 0.7211843, 0.71726274, 0.71770984...",1
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,"[0.1767098, 0.20848627, 0.23225097, 0.2115804,...",1


In [0]:
another=[]
for i in range(len(final_dataset)):
    another.append(final_dataset.X[i].reshape(64,64,1))

#**Binary Face Classifier**

In [0]:
X=np.array(another)
y=final_dataset.y.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True, test_size=1/3, random_state=8613)
del model
model=Sequential()
model.add(Conv2D(32, kernel_size=(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(strides=(2,2)))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adadelta',loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)
pred_train=np.round(model.predict(X_train))
pred_test=np.round(model.predict(X_test))
f1_score(y_test,pred_test)

Train on 3850 samples, validate on 1925 samples
Epoch 1/5
3850/3850 [==============================] - 2s 531us/step - loss: 0.3996 - acc: 0.8169 - val_loss: 0.2096 - val_acc: 0.9314
Epoch 2/5
3850/3850 [==============================] - 2s 403us/step - loss: 0.1459 - acc: 0.9540 - val_loss: 0.0989 - val_acc: 0.9735
Epoch 3/5
3850/3850 [==============================] - 2s 410us/step - loss: 0.0867 - acc: 0.9766 - val_loss: 0.0757 - val_acc: 0.9813
Epoch 4/5
3850/3850 [==============================] - 2s 418us/step - loss: 0.0532 - acc: 0.9888 - val_loss: 0.0554 - val_acc: 0.9844
Epoch 5/5
3850/3850 [==============================] - 2s 423us/step - loss: 0.0398 - acc: 0.9917 - val_loss: 0.0713 - val_acc: 0.9740


0.9747474747474748

In [0]:
# model.save('initialized_model_g.model')

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 30753     
Total params: 31,073
Trainable params: 31,073
Non-trainable params: 0
_________________________________________________________________


# **Transfer Learning**

In [0]:
X = pd.read_csv('data/X.csv',sep=' ',header=None, dtype=float)
y = pd.read_csv('data/y_bush_vs_others.csv',header=None)
y_bush=y.values.ravel()
X = X.as_matrix().reshape(13233,64,64,1)
X_train, X_test, y_train, y_test = train_test_split(X, y_bush,shuffle=True, test_size=1/3, random_state=8613,stratify=y_bush)

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)
pred_train=np.round(model.predict(X_train))
pred_test=np.round(model.predict(X_test))
print('Train F1 Score ', f1_score(y_train, pred_train))
print('Test F1 Score ', f1_score(y_test, pred_test))

Train on 8822 samples, validate on 4411 samples
Epoch 1/20
8822/8822 [==============================] - 4s 424us/step - loss: 0.2486 - acc: 0.9421 - val_loss: 0.1737 - val_acc: 0.9594
Epoch 2/20
8822/8822 [==============================] - 4s 444us/step - loss: 0.1519 - acc: 0.9569 - val_loss: 0.1173 - val_acc: 0.9633
Epoch 3/20
8822/8822 [==============================] - 4s 427us/step - loss: 0.1211 - acc: 0.9624 - val_loss: 0.0969 - val_acc: 0.9667
Epoch 4/20
8822/8822 [==============================] - 4s 433us/step - loss: 0.0938 - acc: 0.9704 - val_loss: 0.0839 - val_acc: 0.9732
Epoch 5/20
8822/8822 [==============================] - 4s 425us/step - loss: 0.0798 - acc: 0.9722 - val_loss: 0.0808 - val_acc: 0.9712
Epoch 6/20
8822/8822 [==============================] - 4s 419us/step - loss: 0.0652 - acc: 0.9779 - val_loss: 0.0719 - val_acc: 0.9778
Epoch 7/20
8822/8822 [==============================] - 4s 432us/step - loss: 0.0563 - acc: 0.9814 - val_loss: 0.0795 - val_acc: 0.9723


In [0]:
model.save('initialized_model_bush_773.model')                                                                                                                                     